In [ ]:
!pip install transformers
!pip install nltk

In [ ]:
from transformers import pipeline
import numpy as np
from transformers import AutoTokenizer

In [ ]:
tokenizer_1 = AutoTokenizer.from_pretrained("Falconsai/text_summarization")
tokenizer_2 = AutoTokenizer.from_pretrained("Neupane9Sujal/Text_Summarization")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
models = {
    "model_1": {"pipeline": pipeline("summarization",model="Falconsai/text_summarization"), "tokenizer": tokenizer_1},
    "model_2": {"pipeline": pipeline("summarization", model="Neupane9Sujal/Text_Summarization"),"tokenizer":tokenizer_2},

}

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
evaluation_metrics = ["bleu", "rouge"]
weights = [0.5, 0.5]

In [ ]:
model_scores = {model_name: {} for model_name in models.keys()}

In [ ]:
input_paragraph = "However, there has been no immediate reaction from Jordan, a kingdom bordering Iraq, Israel, the Palestinian territory of the West Bank, Saudi Arabia, and Syria.U.S. troops long have used Jordan, a close American ally. as a basing point. Some 3,000 American troops typically are stationed in Jordan.While we are still gathering the facts of this attack, we know it was carried out by radical Iran-backed militant groups operating in Syria and Iraq, Biden said in his statement, adding, The US will carry on their commitment to fight terrorism.The US troops in Iraq and Syria have faced drone and missile attacks on their bases since Israels war on Hamas in the Gaza Strip began. The attack on Jordan marks the first targeting of American troops in Jordan during the war."

In [ ]:
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=7548f23be33caf586a7997da813d23f1584ac38b5f90ac7e3562766db596fe82
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
from sklearn.preprocessing import MinMaxScaler
from nltk.translate.bleu_score import sentence_bleu

In [ ]:

scaler = MinMaxScaler()

for model_name, model_info in models.items():
    generated_summary = model_info["pipeline"](input_paragraph, max_length=50, min_length=10, do_sample=False)[0]["summary_text"]

    # Use a more representative reference summary if available
    reference_summary = "Meaningful reference summary for evaluation."  # Replace with actual reference

    # Evaluate BLEU score (ensure correct implementation of sentence_bleu)
    bleu_score = sentence_bleu([reference_summary.split()], generated_summary.split())

    # Evaluate ROUGE score
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_score = rouge_scorer_obj.score(reference_summary, generated_summary)

    model_scores[model_name]["bleu_score"] = bleu_score
    model_scores[model_name]["rouge_score"] = rouge_score

# Normalize scores (handle potential NaN/Inf values)
normalized_scores = scaler.fit_transform([[score['bleu_score'], score['rouge_score']['rouge1'].fmeasure, score['rouge_score']['rougeL'].fmeasure] for score in model_scores.values()])
normalized_scores = np.nan_to_num(normalized_scores)

# Check for NaN or infinite values in normalized scores
if np.isnan(normalized_scores).any() or np.isinf(normalized_scores).any():
    print("Normalized scores contain NaN or infinite values. Check normalization process.")

# Calculate ideal and negative ideal solutions
ideal_solution = np.max(normalized_scores, axis=0)
negative_ideal_solution = np.min(normalized_scores, axis=0)

# Check if ideal and negative ideal solutions are the same
if np.array_equal(ideal_solution, negative_ideal_solution):
    print("Ideal and negative ideal solutions are the same. Check normalization process.")

# Calculate distances to ideal and negative ideal solutions
dist_to_ideal_solution = np.linalg.norm(normalized_scores - ideal_solution, axis=1)
dist_to_negative_ideal_solution = np.linalg.norm(normalized_scores - negative_ideal_solution, axis=1)

# Check if distances are all zero
if np.allclose(dist_to_ideal_solution, 0) and np.allclose(dist_to_negative_ideal_solution, 0):
    print("Distances to ideal and negative ideal solutions are all zero. Check normalization or evaluation process.")

# Calculate relative closeness, handle zero division or very small values
epsilon = 1e-6  # Small epsilon value
relative_closeness = dist_to_negative_ideal_solution / (dist_to_ideal_solution + dist_to_negative_ideal_solution + epsilon)

# Select the best model
best_model_index = np.argmax(relative_closeness)
best_model_name = list(models.keys())[best_model_index]
best_model_score = relative_closeness[best_model_index]

print("Best Model:", best_model_name)




Ideal and negative ideal solutions are the same. Check normalization process.
Distances to ideal and negative ideal solutions are all zero. Check normalization or evaluation process.
Best Model: model_1
